In [1]:
import json

In [20]:
data_name = 'tedlium2'
recog_set = ['train','dev', 'test']
setting = ['noLM','withLM']
nbest = 50

In [21]:
for s in setting:
    for r in recog_set:
        print(f'{s}:{r}')
        json_name = f'./data/{data_name}/raw/{s}/{r}/data.json'
        with open(json_name, 'r') as f,\
            open(f'./data/{data_name}/raw/{s}/{r}/hyp.trn', 'w') as hyp,\
            open(f'./data/{data_name}/raw/{s}/{r}/ref.trn', 'w') as ref:
            j = json.load(f)
            correct = 0
            wrong = 0
            for k in j['utts'].keys():
                count = 0
                # assert(len(j['utts'][k]['output']) == nbest)
                for i, h in enumerate(j['utts'][k]['output']):
                    count += 1
                    # print(j['utts'][k]['output'])
                    seq = [t for t in h['rec_token'].split()]
                    hyp_seq = " ".join(seq[:-1])
                    hyp.write( f'{hyp_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )
                    # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')

                    ref_seq = h["token"]
                    ref.write( f'{ref_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )
                if (count == nbest):
                    correct += 1
                else:
                    wrong += 1
            print(f'correct:{correct}')
            print(f'wrong:{wrong}')

noLM:train
correct:92788
wrong:3
noLM:dev
correct:507
wrong:0
noLM:test
correct:1155
wrong:0
withLM:train
correct:92731
wrong:60
withLM:dev
correct:1152
wrong:3
withLM:test
correct:506
wrong:1


上面code跑完 執行espnet的
```sclite -r ./ref.trn -h  ./hyp.trn -i rm -o all stdout >  ./result.txt```



如果是tedlium2 or librispeech:
```
 /mnt/disk3/Alfred/espnet/utils/spm_decode --model ../../../lang_char/train_trim_unigram500.model --input_format=piece < ./ref.trn | sed -e "s/▁/ /g" > ./ref.wrd.trn
 /mnt/disk3/Alfred/espnet/utils/spm_decode --model ../../../lang_char/train_trim_unigram500.model --input_format=piece < ./hyp.trn | sed -e "s/▁/ /g" > ./hyp.wrd.trn
sclite -r ./ref.wrd.trn trn -h ./hyp.wrd.trn trn -i rm -o all stdout > ./result.wrd.txt
```

In [40]:
data_name = 'tedlium2'
setting = ['noLM','withLM']
recog_set = ['dev', 'test']
nbest = 50
check_dataset = True

In [42]:
file_type = '.txt'
trn_name = '.trn'
if (data_name in ['tedlium2', 'librispeech'] and check_dataset):
    file_type = ".wrd.txt"
    trn_name = '.wrd.trn'

print(file_type)
for s in setting:
    for task in recog_set:
        nbest_err = []
        nbest_num = []
        print(f'{s}:{task}')
        with open(f'./data/{data_name}/raw/{s}/{task}/data.json', 'r') as f:
            json_data = json.load(f)
            for k in json_data['utts'].keys():
                nbest_num.append(len(json_data['utts'][k]['output']))
            print(f'len of best_num:{len(nbest_num)}')
        
        with open(f'./data/{data_name}/raw/{s}/{task}/result{file_type}', 'r') as f, \
             open(f'./data/{data_name}/raw/{s}/{task}/ref{trn_name}', 'r') as rf,\
             open(f'./data/{data_name}/raw/{s}/{task}/hyp{trn_name}', 'r') as hf :
            temp_err = []
            nbest_index = 0
            for i, line in enumerate(f):
                if ('Scores:' in line):
                    scores = line.split()[-4:]
                    scores = [int(s) for s in scores]
                    temp_err.append(scores)
                    if (len(temp_err) == nbest_num[nbest_index]):
                        nbest_err.append(temp_err)
                        temp_err = []
                        nbest_index += 1

            nbest_token = []
            temp_token = []
            nbest_index = 0
            for i, line in enumerate(hf):
                parenthesis_index = line.find('(')
                token = line[ : parenthesis_index]
                temp_token.append(token.strip())

                if (len(temp_token) == nbest_num[nbest_index]):
                    nbest_token.append(temp_token)
                    temp_token = []
                    nbest_index += 1

            ref_text = []
            nbest_index = 0
            line_count = 0
            for i, line in enumerate(rf):
                line_count += 1
                if (line_count == nbest_num[nbest_index]):
                    parenthesis_index = line.find('(')
                    token = line[ : parenthesis_index]
                    ref_text.append(token.strip())
                    nbest_index += 1
                    line_count = 0

            print(f'len of err:{len(nbest_err)}')

            names = []
            nbest_am_score = []
            nbest_ctc_score = []
            nbest_score = []

            if (s == 'withLM'):
                nbest_lm_score = []

            print(f"len:{len(json_data['utts'].keys())}")
            for k in json_data['utts'].keys():
                am_score = []
                ctc_score = []
                score = []

                if (s == 'withLM'):
                    lm_score = []
                    
                for i, h in enumerate(json_data['utts'][k]['output']):
                    score.append(h['score'])
                    am_score.append(h['am_score'])
                    ctc_score.append(h['ctc_score'])

                    if (s == 'withLM'):
                        lm_score.append(h['lm_score'])

                nbest_score.append(score)
                nbest_am_score.append(am_score)
                nbest_ctc_score.append(ctc_score)

                if (s == 'withLM'):
                    nbest_lm_score.append(lm_score)

                names.append(k)
            dataset = []
            if (s != 'withLM'):
                for name, token, score, am_score, ctc_score, err, ref in zip(
                    names, 
                    nbest_token, 
                    nbest_score,
                    nbest_am_score,
                    nbest_ctc_score, 
                    nbest_err, 
                    ref_text
                ):
                    dataset.append(
                        {
                            'name': name,
                            'hyp': token,
                            'am_score': am_score,
                            'ctc_score': ctc_score,
                            'score': score,
                            'err': err,
                            'ref': ref,
                        }
                    )

            else:
                for name, token, score, am_score, ctc_score, lm_score ,err, ref in zip(
                    names, 
                    nbest_token, 
                    nbest_score,
                    nbest_am_score,
                    nbest_ctc_score,
                    nbest_lm_score,
                    nbest_err, 
                    ref_text
                ):
                    dataset.append(
                        {
                            'name': name,
                            'hyp': token,
                            'am_score': am_score,
                            'ctc_score': ctc_score,
                            'lm_score': lm_score,
                            'score': score,
                            'err': err,
                            'ref': ref,
                        }
                    )
            print(f"len of dataset:{len(dataset)}")

        with open(f"./data/{data_name}/data/{s}/{task}/data.json", 'w') as f:
            json.dump(dataset, f, ensure_ascii=False, indent = 4)

.wrd.txt
noLM:dev
len of best_num:507
len of err:507
len:507
len(nbest_token):507
len(ref_text):507
len(nbest_score):507
len(nbest_am_score):507
len(nbest_ctc_score):507
len of dataset:507
noLM:test
len of best_num:1155
len of err:1155
len:1155
len(nbest_token):1155
len(ref_text):1155
len(nbest_score):1155
len(nbest_am_score):1155
len(nbest_ctc_score):1155
len of dataset:1155
withLM:dev
len of best_num:507
len of err:507
len:507
len(nbest_token):507
len(ref_text):507
len(nbest_score):507
len(nbest_am_score):507
len(nbest_ctc_score):507
len of dataset:507
withLM:test
len of best_num:1155
len of err:1155
len:1155
len(nbest_token):1155
len(ref_text):1155
len(nbest_score):1155
len(nbest_am_score):1155
len(nbest_ctc_score):1155
len of dataset:1155


## Result trn

In [8]:
dataname = 'aishell'
name = "Audio_Aware"
setting = 'noLM'
recog_set = ['dev', 'test']
nbest = 3
best_type = 'align_concat'
training = 'MD'

# BART

In [9]:
for r in recog_set:
    json_name = f"./src/Correction/data/{dataname}/{setting}/{r}/{nbest}{best_type}/correct_data.json"
    with open(json_name, 'r') as f, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/{nbest}{best_type}/hyp.trn', 'w') as hyp, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/{nbest}{best_type}/ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['hyp']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['ref']
            ref.write( f'{ref_seq} ({k})\n' )

# RescoreBert

In [11]:
for task in recog_set:
    json_name = f"./data/{dataname}/{task}/{training}/{setting}/{nbest}best_rescore_data.json"
    with open(json_name, 'r') as f, \
        open(f'./data/{dataname}/{task}/{training}/{setting}/{nbest}best_hyp.trn', 'w') as hyp, \
        open(f'./data/{dataname}/{task}/{training}/{setting}/{nbest}best_ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['output']['rec_token']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['output']['text_token']
            ref.write( f'{ref_seq} ({k})\n' )
    print(f'sclite -r ./{nbest}best_hyp.trn -h  ./{nbest}best_ref.trn -i rm -o all stdout >  ./{nbest}best_result.txt')

sclite -r ./50best_hyp.trn -h  ./50best_ref.trn -i rm -o all stdout >  ./50best_result.txt
sclite -r ./50best_hyp.trn -h  ./50best_ref.trn -i rm -o all stdout >  ./50best_result.txt


In [ ]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/{name}/rescore_data.json'
    with open(json_name, 'r') as f, open(f'./data/{dataset}_{r}/{name}/hyp_rescore.trn', 'w') as hyp, open(f'./data/{dataset}_{r}/{name}/ref_rescore.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['rec_text']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['ref_text']
            ref.write( f'{ref_seq} ({k})\n' )

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
import json
import torch
# for r in recog_set:
best_weight = 0
best_cer = 100
with open(f"./data/{dataname}/dev/{nbest}best_{training}_rescore_data_{setting}.json") as f:
    data = json.load(f)
    
    for n in range(101):
        weight = n * 0.01
        c = 0
        s = 0
        de = 0
        i = 0
        for d in data:
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            c += cer[max_index][0]
            s += cer[max_index][1]
            de += cer[max_index][2]
            i += cer[max_index][3]
 
        cer = (s + de + i) / (c + s + de)

        if (cer < best_cer):
            best_cer = cer
            best_weight = weight

print(f'best weight:{best_weight} with cer:{best_cer}')

for task in recog_set:
    with open(f"./data/{dataname}/{task}/{nbest}best_{training}_rescore_data_{setting}.json", 'r') as f, \
         open(f'./data/{dataname}/{task}/hyp_mlm.trn', 'w') as h,\
         open(f'./data/{dataname}/{task}/ref_mlm.trn', 'w') as g:
        data = json.load(f)
        for n, d in enumerate(data):
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
            ref = d['ref'][5:-5]
            token = d['token']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            best_hyp = token[max_index]
            sep = best_hyp.index(102)
            hyp_seq = tokenizer.convert_ids_to_tokens(best_hyp[1:sep])

            h.write( f'{" ".join(hyp_seq)} ({r}_{n})\n' )
            g.write( f'{" ".join(list(ref))} ({r}_{n})\n')
